In [1]:
import numpy as np
import re
import pandas as pd
import string
import math

In [37]:
strings=["hello world","hello this is a sentence","hello this below functions are good to run for medium sized datasets","world punc.!tua:t'ion &se'n-ten?ce"]
data=pd.DataFrame({'data':strings})

In [38]:
#ANCHOR - text cleaning function
def cleantext(text):
    #STUB - string_clean=['<br />','"',',',':','!','&','(',')','?','$','-',';','/'] ---> list for using with a loop
    #NOTE -  the next lines are based on your requirements based on your data
    text=text.lower()
    text=text.replace('<br />',' ')
    text=re.sub(r'[.*<>!&()"?$-/:=1234567890]','',text)
    text=re.sub(r"[']",'',text)
    text=" ".join(text.split())
    return text

totalclean = lambda text: cleantext(text)#TODO - use this on a pandas dataframe by using data.apply()

In [ ]:
#TODO - you can remove stop words and add lemmatisers or stemmers here for this step

In [39]:
data['data']=data['data'].apply(totalclean)

In [40]:
#ANCHOR - unique word mapper funtion
def unique_words_tokeniser(data):#pass the entire dataframe into tis function
    #NOTE - collect all the unique words and map to a token using dictionary method
    unique_word_map={}
    click=0
    for text in data:
        for word in text.split():
            if word not in unique_word_map:
                unique_word_map[word]=click
                click=click+1
    return unique_word_map

In [41]:
unique_word_map=unique_words_tokeniser(data['data'])

In [42]:
#TODO - before using this function create unique_word_map
#ANCHOR - text tokeniser for a single entry
def sent_tokeniser(text,unique_word_map):
    list_word=text.split()
    for i in range(len(list_word)):
        list_word[i]=unique_word_map[list_word[i]]
    return list_word

tokenise=lambda text: sent_tokeniser(text,unique_word_map)#TODO - use this on a pandas dataframe by using data.apply()

In [43]:
data['token']=data['data'].apply(tokenise)

In [44]:
#REVIEW - due to memory issue changed this vector to a sparse matrix using a dictionary
#[2,1,0,0,5,0,0,0,0,0]===>{0:2,1:1,4:5}
def count_vectoriser(text):
    dict1={}
    for num in text:
        if num in dict1:
            dict1[num]=dict1[num]+1
        else:
            dict1[num]=1
    return dict1

vectorise=lambda text: count_vectoriser(text)#TODO - use this on a pandas dataframe by using data.apply()

In [45]:
data['vector']=data['token'].apply(vectorise)

In [46]:
#ANCHOR - sparse matrix creation from vector and dictionary being passed
def sparse_matrix(input,type):
    spm=[]
    if type=="vector":
        for i in range(len(input)):
            if not input[i]==0:
                spm.append([i,input[i]])
        
    elif type=="dict":
        for key in input:
            if not input[key]==0:
                spm.append([key,input[key]])
    else:
        raise TypeError
        print("input type not specified")
    return np.array(spm)

In [47]:
#NOTE - consumes huge time time for bigger dataset
def all_counts(data):
    unique={}
    df_count={}
    for text in data:
        for word in text.split():
            if word in unique:
                unique[word]=unique[word]+1
            else:
                unique[word]=1

    for word in unique:
        df_count[word]=0

    for text in data:
        for word in unique:
            if word in text.split():
                df_count[word]=df_count[word]+1
            else:
                continue

    return unique,df_count

r,t=all_counts(strings)
print(r,t)

{'hello': 3, 'world': 2, 'this': 2, 'is': 1, 'a': 1, 'sentence': 1, 'below': 1, 'functions': 1, 'are': 1, 'good': 1, 'to': 1, 'run': 1, 'for': 1, 'medium': 1, 'sized': 1, 'datasets': 1, "punc.!tua:t'ion": 1, "&se'n-ten?ce": 1} {'hello': 3, 'world': 2, 'this': 2, 'is': 1, 'a': 1, 'sentence': 1, 'below': 1, 'functions': 1, 'are': 1, 'good': 1, 'to': 1, 'run': 1, 'for': 1, 'medium': 1, 'sized': 1, 'datasets': 1, "punc.!tua:t'ion": 1, "&se'n-ten?ce": 1}


In [48]:
#REVIEW - document frequency can also be written using numpy array but here it is written using dictionary
#ANCHOR - document frequency and n function 
def df_n(data):
    df={}
    n={}
    for vector in data:
        for key in vector:
            if key in df:
                df[key]=df[key]+1
                n[key]=n[key]+vector[key]
            else:
                df[key]=1
                n[key]=vector[key]
    return df,n

In [49]:
df,n=df_n(data['vector'])

In [61]:
#ANCHOR - inverse document frequency using dictionary
def idf_(df,n):
    idf={}
    for key in n:
        mt=n[key]/(1+df[key])+1
        idf[key]=round(math.log(mt),4)
    return idf

In [62]:
idf=idf_(df,n)

In [63]:
#ANCHOR - tfidf given in dictionary format applied to the vector
def tfidf(vector):
    tfidf={}
    for tok in vector:
        tfidf[tok]=idf[tok]*vector[tok]
    return tfidf

transform=lambda vector: tfidf(vector)#TODO - use on a pandas dataframe

In [64]:
data['tfidf']=data['vector'].apply(transform)

In [65]:
data.head()

,data,token,vector,tfidf
0,hello world,"[0, 1]","{0: 1, 1: 1}","{0: 0.5596, 1: 0.5108}"
1,hello this is a sentence,"[0, 2, 3, 4, 5]","{0: 1, 2: 1, 3: 1, 4: 1, 5: 1}","{0: 0.5596, 2: 0.5108, 3: 0.4055, 4: 0.4055, 5..."
2,hello this below functions are good to run for...,"[0, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","{0: 1, 2: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11...","{0: 0.5596, 2: 0.5108, 6: 0.4055, 7: 0.4055, 8..."
3,world punctuation sentence,"[1, 16, 5]","{1: 1, 16: 1, 5: 1}","{1: 0.5108, 16: 0.4055, 5: 0.5108}"


In [55]:
print(unique_word_map)

{'hello': 0, 'world': 1, 'this': 2, 'is': 3, 'a': 4, 'sentence': 5, 'below': 6, 'functions': 7, 'are': 8, 'good': 9, 'to': 10, 'run': 11, 'for': 12, 'medium': 13, 'sized': 14, 'datasets': 15, 'punctuation': 16}
